In [4]:
import numpy as np
def pad_sequences(sequences, maxlen=10, dtype='int32',
                  padding='pre', truncating='post', value=-1):
    
    if not hasattr(sequences, '__len__'):
        raise ValueError('`sequences` must be iterable.')
    lengths = []
    for x in sequences:
        if not hasattr(x, '__len__'):
            raise ValueError('`sequences` must be a list of iterables. '
                             'Found non-iterable: ' + str(x))
        lengths.append(len(x))

    num_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    x = (np.ones((num_samples, maxlen) + sample_shape) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if not len(s):
            continue  # empty list/array was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError('Truncating type "%s" not understood' % truncating)

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError('Shape of sample %s of sequence at position %s is different from expected shape %s' %
                             (trunc.shape[1:], idx, sample_shape))

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return x


In [5]:
X = [[0,1,0,1,2], [1,2,4,0,0,1,2], [0,1,2]]
y = [[0,1],[0,0],[0,1],[0,1],[0,0]], [[0,1],[1,1],[0,0],[1,1],[0,0],[1,0],[0,1]], [[0,1],[1,1],[0,0]]

In [9]:
pad_sequences(X, maxlen=2, value=-1, truncating='pre')

array([[1, 2],
       [1, 2],
       [1, 2]], dtype=int32)

In [38]:
x = 'this is anmol sharma and ben'

y = [['this'], [' is '], ['anmo']]

splits = []
dlength = 4
# count = len(x) / 4
# for i in range(count):
#     splits.append(x[i*dlength : (i+1)*dlength])
q = x.split()

def label(word):
    wlen = len(word)
    assert(wlen)
    labels = []
    labels = [(word[0], 'B')]
    if len(word) > 1:
        for i in range(1, wlen-1):
            labels.append((word[i], 'M'))
        labels.append((word[wlen-1], 'E'))
    else:
        labels = [(word[0], 'S')]
    return labels
        
def labelsentence(wordlist):
    return [label(word) for word in wordlist]

def ngramlabel(labelledlist, n):
    # list of letter, label
    merged = []
    for l in labelledlist:
        merged += l
    count = len(merged)/n
    ngrammedlist = []
    for i in range(count):
        ngrammedlist.append( merged[i*n : (i+1)*n])
    return ngrammedlist    
    
    


In [39]:
q=labelsentence(x.split())
ngramlabel(q,2)



[[('t', 'B'), ('h', 'M')],
 [('i', 'M'), ('s', 'E')],
 [('i', 'B'), ('s', 'E')],
 [('a', 'B'), ('n', 'M')],
 [('m', 'M'), ('o', 'M')],
 [('l', 'E'), ('s', 'B')],
 [('h', 'M'), ('a', 'M')],
 [('r', 'M'), ('m', 'M')],
 [('a', 'E'), ('a', 'B')],
 [('n', 'M'), ('d', 'E')],
 [('b', 'B'), ('e', 'M')]]

In [40]:
q

[[('t', 'B'), ('h', 'M'), ('i', 'M'), ('s', 'E')],
 [('i', 'B'), ('s', 'E')],
 [('a', 'B'), ('n', 'M'), ('m', 'M'), ('o', 'M'), ('l', 'E')],
 [('s', 'B'), ('h', 'M'), ('a', 'M'), ('r', 'M'), ('m', 'M'), ('a', 'E')],
 [('a', 'B'), ('n', 'M'), ('d', 'E')],
 [('b', 'B'), ('e', 'M'), ('n', 'E')]]

In [55]:
# we open the count file, get the word and assign labels to each character
# cannot use dictionaries, since the same character may appear again and overwrites the value at its place in dict.
def generateTupleList(filename='/local-scratch/asa224/word_file_1M'):
    with(open(filename, 'rb')) as f:
        label = []
        for line in f:
            word, count = line.split('\t')

            # making sure the parsing is going fine
            assert int(count) == 0

            word = unicode(word, 'utf-8')
            if len(word) == 1:
                label.append((word[0], 3))
            else:
                for i, character in enumerate(word):
                    if i == 0: # this is the first letter
                        label.append((character, 0))
                    elif i == (len(word) - 1): # this is the last letter
                        label.append((character, 2))
                    else: # this is somewhere in the middle
                        label.append((character, 1))

        f.close()
        return label
    
def nGramSequenceGenerator(labelledlist, n):
    """
    Takes as input the label list of tuples generated by the code above. 
    The function generates sequence of size "n" from the given list. 
    """
    count = len(labelledlist)/n
    ngrammedlist = []
    for i in range(count):
        ngrammedlist.append( labelledlist[i*n : (i+1)*n])
    return ngrammedlist   

In [56]:
label

[(u'\u7531', 0),
 (u'\u4e8e', 2),
 (u'\u5927', 0),
 (u'\u9646', 2),
 (u'\u8d74', 3),
 (u'\u53f0', 3),
 (u'\u65c5', 0),
 (u'\u6e38', 2),
 (u'\u8003', 0),
 (u'\u5bdf', 2),
 (u'\u56e2', 3),
 (u'\u7684', 3),
 (u'\u884c', 0),
 (u'\u7a0b', 1),
 (u'\u5b89', 1),
 (u'\u6392', 2),
 (u'\u5341', 0),
 (u'\u5206', 2),
 (u'\u7d27', 0),
 (u'\u51d1', 2),
 (u'\uff0c', 3),
 (u'\u56e2', 0),
 (u'\u5458', 1),
 (u'\u4eec', 2),
 (u'\u66f4', 0),
 (u'\u591a', 1),
 (u'\u7684', 2),
 (u'\u65f6', 0),
 (u'\u5019', 2),
 (u'\u53ea', 3),
 (u'\u80fd', 3),
 (u'\u662f', 3),
 (u'\u5728', 3),
 (u'\u65c5', 0),
 (u'\u884c', 2),
 (u'\u8f66', 3),
 (u'\u4e0a', 3),
 (u'\u8d70', 0),
 (u'\u9a6c', 1),
 (u'\u89c2', 1),
 (u'\u82b1', 1),
 (u'\u5730', 2),
 (u'\u8003', 0),
 (u'\u5bdf', 2),
 (u'\u53f0', 0),
 (u'\u5317', 2),
 (u'\u5e02', 3),
 (u'\u5bb9', 3),
 (u'\uff0c', 3),
 (u'\u6240', 0),
 (u'\u4ee5', 2),
 (u'\u4eba', 0),
 (u'\u6c11', 1),
 (u'\u5e01', 2),
 (u'\u4e0a', 3),
 (u'\u5468', 0),
 (u'\u7684', 2),
 (u'\u8d2c', 0),
 (u'\u503c', 2

In [87]:
def nGramSequenceGenerator(labelledlist, n):
    count = len(labelledlist)/n
    ngrammedlist = []
    for i in range(count):
        ngrammedlist.append( labelledlist[i*n : (i+1)*n])
    return ngrammedlist   

In [88]:
nGramSequenceGenerator(label[0:10], n=2)

[[(u'\u7531', 0), (u'\u4e8e', 2)],
 [(u'\u5927', 0), (u'\u9646', 2)],
 [(u'\u8d74', 3), (u'\u53f0', 3)],
 [(u'\u65c5', 0), (u'\u6e38', 2)],
 [(u'\u8003', 0), (u'\u5bdf', 2)]]

In [89]:
nGramSequenceGenerator(label[0:12], n=3)

[[(u'\u7531', 0), (u'\u4e8e', 2), (u'\u5927', 0)],
 [(u'\u9646', 2), (u'\u8d74', 3), (u'\u53f0', 3)],
 [(u'\u65c5', 0), (u'\u6e38', 2), (u'\u8003', 0)],
 [(u'\u5bdf', 2), (u'\u56e2', 3), (u'\u7684', 3)]]